# Vehicle Sales Price Predictions Workshop - Part 3 of 3


## Inference Pipeline

In order to make a machine learning system from this dataset, we have structured the service into 3 pipelines:

1. feature engineering pipeline notebook (see Part 1)
2. training pipeline notebook (see Part 2)
3. inferencing pipeline notebook (this Part 3)

This notebook will outline the third step, ie. the inference pipeline.

In [1]:
# We will use the Hopsorks Model Registry to instantiate the Model

import hopsworks
import joblib
import torch
import torch.nn as nn

proj = hopsworks.login()
fs = proj.get_feature_store()
mr = proj.get_model_registry()

feature_view = fs.get_feature_view("car_prices_pytorch", version=1)

model = mr.get_model(
    "car_prices_pytorch",
    version=1,
)

# Download the model directory from the Model Registry
model_dir = model.download()

# Load the model using joblib from the downloaded model directory
label_encoders = joblib.load(model_dir + "/label_encoders.pkl")

# Definition of the model
class DeepRegressor(nn.Module):
    def __init__(self, input_size):
        super(DeepRegressor, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Load the file of the model
model_data = torch.load(model_dir + '/regression_model.pth', map_location=torch.device('cpu'))

Connected. Call `.close()` to terminate connection gracefully.

Multiple projects found. 

	 (1) Car_Prices
	 (2) GraphEmbeddingsDemo
	 (3) rixdemo
	 (4) BeerVolumePrediction

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/818324
2024-07-04 11:50:41,470 WARNING: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead

2024-07-04 11:50:41,471 WARNING: using legacy validation callback
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


In [2]:
# Check the type of the loaded object
if isinstance(model_data, dict) and 'model_state_dict' in model_data:
    state_dict = model_data['model_state_dict']
elif isinstance(model_data, torch.nn.Module):
    state_dict = model_data.state_dict()
else:
    raise ValueError("The file does not contain a valid PyTorch model or the format is unexpected.")

# Show weights of each layer
print("Show the weights of the different layers of the model:")
for layer_name, weights in state_dict.items():
    print(f"\nLayer: {layer_name}")
    print(f"Weights: {weights.shape}")
    print(weights)

# If the object contains another structure, inspect it
if isinstance(model_data, dict):
    print("\nComplete structure of the saved model:")
    for key, value in model_data.items():
        if key != 'model_state_dict':
            print(f"\nStructure of the key '{key}':")
            if isinstance(value, torch.Tensor):
                print(f"  - Tensor with shape : {value.shape}")
            elif isinstance(value, dict):
                print("  - Dictionnary with keys :")
                for subkey in value.keys():
                    print(f"    - {subkey}")
            else:
                print(f"  - Type : {type(value)}")


# If the object is directly the model, display its architecture
elif isinstance(model_data, torch.nn.Module):
    print("\nThe saved model is a direct PyTorch model.")
    print("Architecture of the model :")
    print(model_data)

Show the weights of the different layers of the model:

Layer: fc1.weight
Weights: torch.Size([64, 10])
tensor([[ 5.9588e-01,  8.0228e-01,  8.1982e-01,  4.5735e-01,  9.6425e-01,
          4.8863e-01,  1.2716e+00, -7.3771e-02,  8.6800e-01,  8.2294e-01],
        [ 2.3588e-01,  1.0326e-01,  2.8148e-01, -2.9166e-01, -1.9072e-01,
          8.4053e-02,  2.0952e-01, -1.9053e+00,  4.6597e-01,  4.3701e-01],
        [-1.0098e-01, -1.5421e-01, -3.4394e-01,  2.0782e-01, -2.7555e-01,
          5.1356e-01, -2.0847e-01, -8.6191e-01,  1.3199e-01,  1.6592e-01],
        [-1.6649e-01, -1.4767e-01,  1.8958e-01,  1.4087e-01, -2.7359e-01,
         -3.0950e-01,  1.7447e-01, -2.6259e-01,  1.7768e-01, -2.2107e-02],
        [-9.5350e-01, -5.6322e-01, -1.7968e-01, -2.4840e-01, -5.2574e-01,
         -1.1003e-01, -1.2163e+00, -2.2283e-03, -5.4540e-01, -2.8267e-01],
        [ 9.2761e-03, -2.3221e-01, -1.7132e-01, -1.5441e-01, -3.7079e-02,
         -3.3038e-01,  1.4254e-02, -6.0565e-02, -5.2801e-01, -1.7376e-01],
  

In [3]:
test_data = feature_view.get_batch_data(start_time="2015-07-01 00:00")
test_data

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.14s) 


,year,make,model,trim,body,transmission,condition,odometer,color,interior
0,2013,Toyota,Tacoma,V6,double cab,manual,27.0,43988.0,green,gray
1,2011,GMC,Yukon XL,Denali,suv,automatic,38.0,29767.0,black,black
2,2007,Chevrolet,Silverado 1500 Classic,LS2,crew cab,automatic,24.0,76663.0,gold,gray
3,2012,Chevrolet,Sonic,LT,Hatchback,manual,36.0,64312.0,brown,gray
4,2013,Dodge,Grand Caravan,SXT,Minivan,automatic,36.0,70859.0,gold,black
...,...,...,...,...,...,...,...,...,...,...
831,2004,Chevrolet,Corvette,Z06,Coupe,manual,41.0,69537.0,yellow,black
832,2007,Volvo,XC90,3.2,suv,automatic,34.0,95769.0,white,beige
833,2012,BMW,1 Series,128i,Coupe,automatic,46.0,25056.0,black,black
834,2014,Ford,E-Series Van,E-250,E-Series Van,automatic,35.0,11467.0,white,gray


In [4]:
from sklearn.preprocessing import LabelEncoder

# Define the function to encode categorical data
def encode_categorical_data(dataset, label_encoders):    
    # Iterate over the columns of the DataFrame
    for column in dataset.columns:
        # Check if the column is of type 'object' (categorical)
        if dataset[column].dtype == 'object':
            # Retrieve already fitted LabelEncoder
            label_encoder = label_encoders[column.lower()]
            
            # Perform encoding on unique column values
            dataset[column] = label_encoder.transform(dataset[column])
    return dataset

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd

# Define the function to predict selling prices
def predict_selling_price(data, label_encoders, model):
        
    # Preprocess the data
    processed_data = encode_categorical_data(data, label_encoders)
    
    # Convert data to PyTorch tensors
    X_test = torch.tensor(processed_data.values.astype(np.float32))
    
    # Pass data to model for prediction
    with torch.no_grad():
        model.eval()
        predictions = model(X_test).numpy()
        
    predictions_df = pd.DataFrame(predictions, columns=['predicted_sale_price'])
    data['predicted_sale_price'] = predictions_df['predicted_sale_price']
    return data

df_encoded = predict_selling_price(test_data, label_encoders, model_data)
df_encoded

,year,make,model,trim,body,transmission,condition,odometer,color,interior,predicted_sale_price
0,2013,45,577,1057,50,1,27.0,43988.0,8,6,9963.453125
1,2011,13,656,429,72,0,38.0,29767.0,1,1,10010.116211
2,2007,7,546,654,45,0,24.0,76663.0,6,6,9756.763672
3,2012,7,558,657,21,1,36.0,64312.0,3,6,9808.267578
4,2013,9,286,958,25,0,36.0,70859.0,6,1,9804.009766
...,...,...,...,...,...,...,...,...,...,...,...
831,2004,7,150,1156,8,1,41.0,69537.0,18,1,9785.300781
832,2007,47,639,167,72,0,34.0,95769.0,17,0,9813.724609
833,2012,3,0,21,8,0,46.0,25056.0,1,1,9848.377930
834,2014,12,171,435,12,0,35.0,11467.0,17,6,9896.207031


### 8. Use your trained model to make price estimates

Make a prediction function to load the model trained and saved in the model_regressor.pth file and then test on data that the user will enter manually. As a reminder, the categorical data was encoded and saved in a label_encoders.pth file. There are also numeric variables that the user must indicate.

In this system we will use [Gradio](https://www.gradio.app/) for creating an easy to use frontend to our machine learning model.

In [8]:
# First we install Gradio
!pip install --quiet gradio==3.48.0

In [9]:
# Second we prepare for the use of Gradio
import gradio as gr
from functools import partial
import warnings
warnings.filterwarnings('ignore')

# Define the method to print out the values entered
def print_values(year, make, model, trim, body, transmission, condition, odometer, color, interior, model_data, label_encoders):
    data = {
        "Year": [year],
        "Make": [make],
        "Model": [model],
        "Trim": [trim],
        "Body": [body],
        "Transmission": [transmission],
        "Condition": [condition],
        "Odometer": [odometer],
        "Color": [color],
        "Interior": [interior]
    }
    df = pd.DataFrame(data)
        
    df_encoded = predict_selling_price(df, label_encoders, model_data)
    return df_encoded.iloc[0]['predicted_sale_price']

print_values_partial = partial(
    print_values, 
    model_data=model_data, 
    label_encoders=label_encoders,
)

In [11]:
# Third we create the Gradio interface
with gr.Blocks() as demo:
    with gr.Row():
        year = gr.Number(label="Year", value=2014)
        make = gr.Dropdown(label="Make", choices=["Toyota", "Ford", "Volkswagen", "BMW"], value="Toyota")
        model = gr.Dropdown(label="Model", choices=["Prius", "Mustang", "Jetta", "X1"], value="Prius")
        trim = gr.Dropdown(label="Trim", choices=["Base", "SL", "GLS", "Luxury"], value="Base")
        body = gr.Dropdown(label="Body", choices=["Hatchback", "Sedan", "SUV", "Minivan"], value="Hatchback")

    with gr.Row():
        transmission = gr.Dropdown(label="Transmission", choices=["automatic", "manual"], value="automatic")
        condition = gr.Number(label="Condition", value=45.0)
        odometer = gr.Number(label="Odometer", value=33761.0)
        color = gr.Dropdown(label="Color", choices=["red", "white", "black", "silver"], value="red")
        interior = gr.Dropdown(label="Interior", choices=["black", "gray", "brown"], value="black")

    submit_button = gr.Button("Submit")
    output = gr.JSON(label="Entered Values")

    submit_button.click(
        print_values_partial,
        inputs=[year, make, model, trim, body, transmission, condition, odometer, color, interior],
        outputs=output,
    )

# Launch the interface
demo.launch(share=True)

Exception in thread Thread-34 (run):
Traceback (most recent call last):
  File "/Users/rvanbruggen/miniforge3/envs/CarPricesEnv/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/Users/rvanbruggen/miniforge3/envs/CarPricesEnv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/Users/rvanbruggen/miniforge3/envs/CarPricesEnv/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/rvanbruggen/miniforge3/envs/CarPricesEnv/lib/python3.12/site-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rvanbruggen/miniforge3/envs/CarPricesEnv/lib/python3.12/site-packages/nest_asyncio.py", line 26, in run
    loop = asyncio.get_event_loop()
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rvanbruggen/miniforge3/envs/CarPricesEnv/lib/pyth

IMPORTANT: You are using gradio version 3.48.0, however version 4.29.0 is available, please upgrade.
--------


Exception in thread Thread-35 (run):
Traceback (most recent call last):
  File "/Users/rvanbruggen/miniforge3/envs/CarPricesEnv/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/Users/rvanbruggen/miniforge3/envs/CarPricesEnv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/Users/rvanbruggen/miniforge3/envs/CarPricesEnv/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/rvanbruggen/miniforge3/envs/CarPricesEnv/lib/python3.12/site-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rvanbruggen/miniforge3/envs/CarPricesEnv/lib/python3.12/site-packages/nest_asyncio.py", line 26, in run
    loop = asyncio.get_event_loop()
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rvanbruggen/miniforge3/envs/CarPricesEnv/lib/pyth

KeyboardInterrupt: 

This completes the entire process of feature engineering, training and inferencing pipelines, and therefore the delivery of an end-to-end machine learning system.

---